In [56]:
# Downloading and Caching Datasets
import hashlib
import os
import tarfile
import zipfile
import requests

#@save
DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'

DATA_HUB['kaggle_house_train'] = (  #@save
    DATA_URL + 'kaggle_house_pred_train.csv',
    '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (  #@save
    DATA_URL + 'kaggle_house_pred_test.csv',
    'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')

def download(name, cache_dir=os.path.join('..', 'data')): #@save 
    # '..'은 parentory directory를 의미한다고 합니다. Now, everything gets clear to me. 자꾸 곱씹어보면 매우 세련된 표현임을 확인할 수 있습니다.
    """Download a file inserted into DATA_HUB, return the local filename."""
    assert name in DATA_HUB, f"{name} does not exist in {DATA_HUB}."
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576)
                if not data: # end of file이 될 때까지 읽는다는 의미인 것 같습니다.
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname # 한번 다운로드가 이루어진 녀석이면 여기에서 끝남을 기억할 필요가 있습니다. CPU에서 요청한 데이터가 캐시에 존재하는 경우를 캐시 히트 (Hit)라고 한다.
    # 이 부분은 처음 파일을 다운로드 받을 때에만 작동되면 함수 영역임을 이해할 수 있습니다.
    print(f'Downloading {fname} from {url}...')
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
        f.write(r.content)
    return fname

# We also implement two additional utility ftns:
# 1) one is to download and extract a zip or tar file
# 2) and the other to download all the datasets used in this book from 'DATA_HUB' into the cache directory.
def download_extract(name, folder=None): #@save
    """Download and extract a zip/tar file."""
    fname = download(name)
    base_dir = os.path.dirname(fname) # fname 이름의 최하단 계층 파일이 속하는 디렉토리명을 반환하는 함수
    # fname 이름을 확장자('ext')와 나머지('data_dir')로 분리함
    data_dir, ext = os.path.splitext(fname)
    if ext == '.zip':
        fp = zipfile.Zipfile(fname, 'r')
    elif ext in ('.tar', '.gz'):
        frp = tarfile.open(fname, 'r')
    else: # 위의 두 경우가 아닐 경우에는 항상 출력되는 문구이다. 압축을 풀 경우가 아닌 경우를 의미합니다.
        assert False, 'Only zip/tar files can be extracted.'
    fp.extractall(base_dir)
    return os.path.join(base_dir, folder) if folder else data_dir # !!!!! 이 부분은 잘 와닿지가 않는데, 이후에 예시가 나오는 부분에서 확인하면 좋을 것 같습니다. !!!!!

# extract해야할 녀석들은 그 확장자의 형태로 다운로드가 될 것입니다. 이렇게 한 다음에 'download_extract()' 함수를 이용하면 될 것 같습니다.
def download_all(): #@save
    """Download all files in the DATA_HUB."""
    for name in DATA_HUB:
        download(name)

In [63]:
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
from d2l import tensorflow as d2l

train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))

print(train_data.shape)
print(test_data.shape)

(1460, 81)
(1459, 80)


In [65]:
print(train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [ ]:
# 'train_data의 column을 indexing할 때 '1:-1'인 이유는 test_data와의 column수를 맞춰주기 위해서입니다.'
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:])) 